In [35]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import torch.nn.functional as F

class BertForSequenceClassification(nn.Module):
    def __init__(self, num_labels):
        super(BertForSequenceClassification, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # print(outputs["last_hidden_state"].shape)
        # print(outputs["pooler_output"].shape)
        pooled_output = outputs[1]  # Take the pooled output (CLS token representation)
        logits = self.classifier(pooled_output)
        probs = self.softmax(logits)
        return probs

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the model
num_labels = 2  # For binary classification, adjust for multi-class
model = BertForSequenceClassification(num_labels)

# Example input text
input_text = "This is an example sentence."

# Tokenize input text
inputs = tokenizer(input_text, return_tensors='pt')
del inputs["token_type_ids"]

# Forward pass
logits = model(**inputs)

print(logits)


tensor([[0.6862, 0.3138]], grad_fn=<SoftmaxBackward0>)


In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset

# Sample data
sample_data = [
    {"text": "I absolutely loved this movie! It was fantastic and well-acted.", "label": 1},
    {"text": "This film was terrible. I wasted two hours of my life.", "label": 0},
    {"text": "An excellent movie with a gripping story and brilliant performances.", "label": 1},
    {"text": "Horrible movie. Poor storyline and bad acting.", "label": 0},
    {"text": "A wonderful experience! I would watch it again.", "label": 1},
    {"text": "Not worth the time. The plot was very predictable.", "label": 0},
    {"text": "Amazing direction and acting. A must-watch!", "label": 1},
    {"text": "The movie was very dull and boring.", "label": 0},
    {"text": "Loved the cinematography and the plot twists.", "label": 1},
    {"text": "The characters were not believable at all.", "label": 0}
]

# Convert to DataFrame
df = pd.DataFrame(sample_data)

# Tokenize data
inputs = tokenizer(df['text'].tolist(), return_tensors='pt', padding=True, truncation=True, max_length=128)

# Convert labels to tensor
labels = torch.tensor(df['label'].tolist())

# Create TensorDataset and DataLoader
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Print sample from DataLoader
for batch in train_loader:
    print(batch)
    break


[tensor([[  101,  1996,  3185,  2001,  2200, 10634,  1998, 11771,  1012,   102,
             0,     0,     0,     0,     0,     0],
        [  101,  2023,  2143,  2001,  6659,  1012,  1045, 13842,  2048,  2847,
          1997,  2026,  2166,  1012,   102,     0]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]), tensor([0, 0])]


In [37]:
import torch
import torch.nn as nn
from transformers import AdamW

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}')

print("Fine-tuning complete!")


Epoch 1/3, Loss: 0.7113
Epoch 2/3, Loss: 0.6900
Epoch 3/3, Loss: 0.6494
Fine-tuning complete!


In [40]:
model.to('cpu')

input_text = "This movie was alright, not quite sure! It could be improved. But good characters!"
inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True, max_length=128)
del inputs["token_type_ids"]

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs[0]

print(logits, logits.shape)
predicted_class = torch.argmax(logits, dim=-1).item()
label_map = {0:"Negative", 1:"Positive"}
predicted_label = label_map[predicted_class]

print(f"Input Text: {input_text}")
print(f"Predicted Label: {predicted_label}")

tensor([0.5379, 0.4621]) torch.Size([2])
Input Text: This movie was alright, not quite sure! It could be improved. But good characters!
Predicted Label: Negative
